In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Install And Imprort Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical

### Read the Dataset

In [3]:
# loading the directories

training_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Training'
test_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Test'

In [4]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [5]:
#image_files

In [6]:
#test_image_files

In [7]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 0


### Read Data and Data Augumentation

In [8]:
# Image Augmentation
from keras.applications import VGG19
from keras.utils import image_dataset_from_directory
from keras.applications.vgg16 import preprocess_input

In [9]:
test_dir

'/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Test'

In [10]:
training_generator = image_dataset_from_directory(training_dir,image_size=(224,224),batch_size =200,label_mode="categorical")
training_generator

NotFoundError: Could not find directory /kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Training

In [ ]:
test_generator = image_dataset_from_directory(test_dir,image_size=(224,224),batch_size =200,label_mode="categorical")
test_generator

### Data processing, Data Augumentation
### Model Building

In [ ]:
from keras.models import Model

IMAGE_SIZE = [224, 224]  # we will keep the image size as (224,224). You can increase the size for better results.

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG19(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(4096, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(1096, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


### Model Training & Testing

In [ ]:
history = model.fit(training_generator,
                   steps_per_epoch = 10000,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results.
                   epochs = 1,  # change this for better results
                   validation_data = test_generator,
                   validation_steps = 3000)  # this should be equal to total number of images in validation set.

### Display Result

In [ ]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))